In [1]:
"""
Multi-threaded script where we separately:
1) Generate an LLM-based AST (two-step approach: CFG + partial block parse),
2) Generate a tree-sitter-based static AST,
3) Compare snippet-level labels,
4) Save both ASTs as JSON.

We split logic into:
- parse_cfg_structure(): returns a high-level "CFG" with classes/functions line ranges
- build_ast_from_cfg(): recursively builds an AST by ignoring function/class lines in the parent block,
  and placing a placeholder for function/class (which we then parse by recursion).
- process_single_file(): calls above, also gets tree-sitter AST, compares, and saves.
Finally main() uses ThreadPoolExecutor to process multiple files in parallel.
"""

import os
import sys
import json
from typing import Any, Dict, List, Tuple
import concurrent.futures
from multiprocessing import cpu_count

from tqdm import tqdm

# LLM interface
from llm import get_llm_answers  # <-- you need a get_llm_answers(prompt, require_json, temperature, model_name, ...) function

# py-tree-sitter
import tree_sitter_python as tspython
from tree_sitter import Language, Parser

###############################################################################
#                          Tree-sitter initialization                         #
###############################################################################
# 假设 tree_sitter_python.language() 可以返回 .so 或 .dylib 路径
PY_LANGUAGE = Language(tspython.language())
parser = Parser(PY_LANGUAGE)

###############################################################################
#                       Tokenization & Utility Functions                      #
###############################################################################
import re

def tokenize_code(code: str) -> List[Tuple[int, int, str]]:
    """
    使用正则分词，直接在原始代码上通过 re.finditer() 获取匹配位置，
    并保留换行符，让 LLM 能识别多行结构。
    """
    token_pattern = (
        r'[A-Za-z_]\w*|'  # 标识符
        r'[0-9]+|'        # 数字
        r'"[^"]*"|'       # 双引号字符串
        r"'[^']*'|"       # 单引号字符串
        r'\\[ntr]|'       # 转义符 \n \t \r
        r'//.*|'          # 单行注释 (如 C++/Java/JS 风格)
        r'/\*.*?\*/|'     # 多行注释 (如 C 风格) - 可选
        r'\n|\r|\t|'      # 处理原生换行/回车/制表符
        r'\S'             # 其他符号(如 +, -, {, }, 以及非空白字符)
    )

    tokens_with_offset = []
    for match in re.finditer(token_pattern, code, re.MULTILINE | re.DOTALL):
        tk = match.group(0)
        start_offset, end_offset = match.span()
        tokens_with_offset.append((start_offset, end_offset, tk))

    return tokens_with_offset

def rebuild_label_llm(
    code: str,
    indexed_tokens: List[Tuple[int, int, str]],
    start_token: int,
    end_token: int
) -> str:
    """
    根据 start_token / end_token 在 indexed_tokens 中查找首尾字符偏移，
    然后从源码中截取原始片段（保留换行）。
    """
    if (
        start_token < 0 or end_token < 0
        or start_token >= len(indexed_tokens)
        or end_token >= len(indexed_tokens)
        or end_token < start_token
    ):
        return ""
    start_offset = indexed_tokens[start_token][0]
    end_offset = indexed_tokens[end_token][1]
    if start_offset < 0 or end_offset > len(code):
        return ""
    snippet = code[start_offset:end_offset]
    return snippet

def collect_llm_labels(node: Dict[str, Any], code: str, indexed_tokens: List[Tuple[int,int,str]]) -> None:
    """
    递归给 LLM AST 中每个节点设置 node["label"] = 源码片段。
    """
    st = node.get("start_token", -1)
    et = node.get("end_token", -1)
    snippet = rebuild_label_llm(code, indexed_tokens, st, et)
    node["label"] = snippet

    for child in node.get("children", []):
        collect_llm_labels(child, code, indexed_tokens)

###############################################################################
#                         Compare: snippet-level text                         #
###############################################################################
def compare_ast_nodes(node1: Dict[str, Any], node2: Dict[str, Any], path: str = ""):
    """
    Compare node["label"] text, plus child count, recursively.
    """
    if not node1 or not node2:
        return

    label1 = (node1.get("label", "") or "").strip()
    label2 = (node2.get("label", "") or "").strip()
    if label1 != label2:
        print(f"[Diff] label mismatch at {path}:")
        print(f"  1: {repr(label1)}")
        print(f"  2: {repr(label2)}")

    c1 = node1.get("children", [])
    c2 = node2.get("children", [])
    if len(c1) != len(c2):
        print(f"[Diff] Child count mismatch at {path}: {len(c1)} vs {len(c2)}")

    for i in range(min(len(c1), len(c2))):
        compare_ast_nodes(c1[i], c2[i], path + f".children[{i}]")

###############################################################################
#               Step A: get CFG structure (class/function line range)         #
###############################################################################
def get_structure_prompt(code_text: str, language: str="python") -> str:
    """
    构造 prompt, 让 LLM 返回:
    {
      "name": "example_script",
      "type": "CFG",
      "start_line": 1,
      "end_line": N,
      "functions": [...],
      "classes": [...]
    }
    """
    lines = code_text.splitlines()
    lines_json = [{"line": i+1, "code": line} for i,line in enumerate(lines)]

    prompt = f"""
You are given a piece of {language} code.
Your task: find all the nested classes and methods in the code,
including their exact start_line and end_line.

Return a JSON with this structure:

{{
  "name": "example_script",
  "type": "CFG",
  "start_line": 1,
  "end_line": {len(lines)},
  "functions": [
    {{
      "name": "function_name",
      "type": "function",
      "start_line": 10,
      "end_line": 20,
      "functions": [],
      "classes": []
    }}
  ],
  "classes": [
    {{
      "name": "class_name",
      "type": "class",
      "start_line": 30,
      "end_line": 50,
      "functions": [
        {{
          "name": "method_name",
          "type": "function",
          "start_line": 35,
          "end_line": 40,
          "functions": [],
          "classes": []
        }}
      ],
      "classes": []
    }}
  ]
}}

IMPORTANT:
1) If a function is nested in a class, put it in the class's "functions".
2) If a class is nested in another class, put it in "classes".
3) start_line/end_line must reflect the actual lines in the original code (1-based).
4) Do not omit or reorder fields. Use the exact structure above.

Here are the code lines:
{json.dumps(lines_json, indent=2)}

Return only valid JSON.
"""
    return prompt

def parse_cfg_structure(code_text: str, language="python") -> Dict[str,Any]:
    """
    调用 LLM, 获取 CFG JSON 结构
    """
    prompt = get_structure_prompt(code_text, language)
    try:
        raw_output = get_llm_answers(prompt, require_json=True, temperature=0, model_name="gpt-4o")
        return json.loads(raw_output)
    except Exception as e:
        print(f"[Error] parse_cfg_structure: {e}")
        # fallback
        return {
          "name": "fallback",
          "type": "CFG",
          "start_line": 1,
          "end_line": len(code_text.splitlines()),
          "functions": [],
          "classes": []
        }

###############################################################################
#        Step B1: LLM-based AST for a snippet (tokens -> AST from LLM)        #
###############################################################################
def llm_build_ast_from_tokens(tokens_with_offset: List[Tuple[int, int, str]], top_level=True) -> Dict[str, Any]:
    """
    给定 tokens列表 => 调用 LLM => 生成 JSON AST.
    - top_level: 是否最外层(只有最外层允许 'module'), 否则用 'block' 等
    """
    indexed_tokens = [(i, t[2]) for i, t in enumerate(tokens_with_offset)]
    token_info = "\n".join(f"{i}: {text}" for (i, text) in indexed_tokens)

    # 构造 prompt
    allowed_types = [
        "aliased_import", "argument_list", "as_pattern", "as_pattern_target", "assert_statement",
        "assignment", "attribute", "augmented_assignment", "await", "binary_operator", "block",
        "boolean_operator", "break_statement", "call", "class_definition", "comment",
        "comparison_operator", "concatenated_string", "conditional_expression",
        "continue_statement", "decorated_definition", "decorator", "default_parameter",
        "delete_statement", "dictionary", "dictionary_comprehension", "dictionary_splat",
        "dictionary_splat_pattern", "dotted_name", "elif_clause", "ellipsis", "else_clause",
        "escape_interpolation", "escape_sequence", "except_clause", "expression_list",
        "expression_statement", "false", "finally_clause", "float", "for_in_clause", "for_statement",
        "format_specifier", "function_definition", "future_import_statement",
        "generator_expression", "generic_type", "global_statement", "identifier", "if_clause",
        "if_statement", "import_from_statement", "import_prefix", "import_statement", "integer",
        "interpolation", "keyword_argument", "keyword_separator", "lambda", "lambda_parameters",
        "line_continuation", "list", "list_comprehension", "list_splat", "list_splat_pattern",
        "module", "named_expression", "none", "nonlocal_statement", "not_operator", "pair",
        "parameters", "parenthesized_expression", "pass_statement", "pattern_list", "raise_statement",
        "relative_import", "return_statement", "set", "set_comprehension", "slice", "string",
        "string_content", "string_end", "string_start", "subscript", "true", "try_statement",
        "tuple", "tuple_pattern", "type", "type_parameter", "typed_default_parameter", "typed_parameter",
        "unary_operator", "union_type", "while_statement", "with_clause", "with_item", "with_statement",
        "yield"
    ]
    allowed_types_str = ", ".join(allowed_types)

    top_level_instruction = "Exactly one 'module' node can appear at the root. Use 'block' if nested.\n"

    prompt = (
        "Below is a list of tokens (index -> token_string) for a code snippet:\n"
        f"{token_info}\n\n"
        "Create a JSON-based AST with these fields:\n"
        f"- 'type': must be in {{{allowed_types_str}}}\n"
        "- 'start_token', 'end_token'\n"
        "- 'children' (array)\n\n"
        "Leaf nodes => start_token == end_token.\n"
        "No overlapping sibling token ranges.\n"
        "Return valid JSON only.\n"
    )
    if top_level:
        prompt += "\nAt the root, use 'module'. Do not nest multiple 'module'.\n" + top_level_instruction
    else:
        prompt += "\nInside blocks, do not produce 'module'. Use 'block' or suitable type.\n" + top_level_instruction

    try:
        llm_output = get_llm_answers(
            prompt,
            model_name="gpt-4o",
            require_json=True,
            temperature=0
        )
        ast_dict = json.loads(llm_output)
        return ast_dict

    except Exception as e:
        print(f"[Error] llm_build_ast_from_tokens: {e}")
        return {
            "type": "ErrorNode",
            "start_token": -1,
            "end_token": -1,
            "children": []
        }

###############################################################################
#        Step B2: 生成sub-AST，排除子function/class行，避免重复解析            #
###############################################################################
def generate_partial_block_ast(
    block_code: str,
    excluded_ranges: List[Tuple[int,int]]
) -> Dict[str,Any]:
    """
    给定一个 block 的代码(多行)，同时给出在该 block 中
    某些行段要排除(因为它们是 class/function 体，另行解析)。
    我们在 block 中把这些行段替换成空行(或注释)，只解析剩余普通语句。
    """
    lines = block_code.splitlines(True)  # 保留换行
    # 假定excluded_ranges已基于"block_code"的局部行号
    for (start_l, end_l) in excluded_ranges:
        # 修正边界
        if start_l < 0: start_l = 0
        if end_l > len(lines): end_l = len(lines)
        for i in range(start_l, end_l):
            lines[i] = ""  # 变成空行

    partial_code = "".join(lines)
    tokens = tokenize_code(partial_code)
    ast_dict = llm_build_ast_from_tokens(tokens, top_level=False)
    # collect label
    all_tokens = tokenize_code(partial_code)
    collect_llm_labels(ast_dict, partial_code, all_tokens)
    return ast_dict

def build_ast_from_cfg(cfg_node: Dict[str,Any], code_lines: List[str]) -> Dict[str,Any]:
    """
    递归地构建 AST:
      1) 先解析当前块 [start_line..end_line] => 但排除 functions/classes 行段
      2) 生成 "父块AST"
      3) 对子function/class 递归 => 并把它们当作一个占位节点插入
    """
    # block行区间
    start_line = cfg_node.get("start_line", 1)
    end_line = cfg_node.get("end_line", len(code_lines))

    # 提取该块的代码
    block_slice = code_lines[start_line-1 : end_line]
    block_code = "".join(block_slice)

    # 找出子function/class行段
    # "excluded_ranges" 在当前block内的相对行号
    excluded = []
    for fn_item in cfg_node.get("functions", []):
        rstart = fn_item["start_line"] - start_line
        rend = fn_item["end_line"] - start_line + 1
        excluded.append((rstart, rend))
    for cl_item in cfg_node.get("classes", []):
        rstart = cl_item["start_line"] - start_line
        rend = cl_item["end_line"] - start_line + 1
        excluded.append((rstart, rend))

    # 1) 父块AST(只解析普通语句)
    block_ast = generate_partial_block_ast(block_code, excluded)

    if "children" not in block_ast:
        block_ast["children"] = []

    # 2) 递归构造子function
    for fn_item in cfg_node.get("functions", []):
        sub_ast = build_ast_from_cfg(fn_item, code_lines)
        block_ast["children"].append({
            "type": "function_placeholder",  # or "function_definition"
            "name": fn_item["name"],
            "start_line": fn_item["start_line"],
            "end_line": fn_item["end_line"],
            "children": [sub_ast]
        })

    # 3) 递归构造子class
    for cl_item in cfg_node.get("classes", []):
        sub_ast = build_ast_from_cfg(cl_item, code_lines)
        block_ast["children"].append({
            "type": "class_placeholder",     # or "class_definition"
            "name": cl_item["name"],
            "start_line": cl_item["start_line"],
            "end_line": cl_item["end_line"],
            "children": [sub_ast]
        })

    return block_ast

def generate_llm_ast_via_cfg(code: str) -> Dict[str,Any]:
    """
    顶层函数:
    1) parse_cfg_structure => 得到 class/func行号
    2) build_ast_from_cfg => 递归构建
    """
    cfg_info = parse_cfg_structure(code, language="python")
    code_lines = code.splitlines(True)
    # 这里若是顶层, 让 "build_ast_from_cfg" -> top_level
    # 也可在 build_ast_from_cfg( top_level=True ), 
    #   但是我们这里在 partial code parse时top_level=true 也可以
    full_ast = build_ast_from_cfg(cfg_info, code_lines)

    # 若你希望顶层节点为 'module', 可以额外再做一次替换:
    if full_ast.get("type") and full_ast["type"] != "module":
        full_ast["type"] = "module"
    return full_ast

###############################################################################
#           Tree-sitter-based static AST                                      #
###############################################################################
class PyTreeSitterStaticHandler:
    def __init__(self):
        self.parser = parser

    def generate_static_ast(self, code: str) -> Dict[str, Any]:
        tree = self.parser.parse(code.encode("utf-8"))
        root_node = tree.root_node
        return self.ts_node_to_dict(root_node)

    def ts_node_to_dict(self, node) -> Dict[str, Any]:
        if not node.is_named:
            return None
        node_type = node.type
        node_text = (node.text or b"").decode("utf-8")
        custom = {
            "type": node_type,
            "label": node_text,
            "children": []
        }
        for i in range(node.child_count):
            child = node.child(i)
            sub = self.ts_node_to_dict(child)
            if sub:
                custom["children"].append(sub)
        return custom

def generate_tree_sitter_ast(code: str) -> Dict[str, Any]:
    handler = PyTreeSitterStaticHandler()
    return handler.generate_static_ast(code)

###############################################################################
#                  process_single_file using CFG-based LLM AST               #
###############################################################################
def process_llm_ast(code: str, file_path: str) -> dict:
    """处理LLM AST生成和保存(使用CFG + partial block parse)"""
    llm_out_dir = "llm_ast/cfg_block"
    os.makedirs(llm_out_dir, exist_ok=True)
    llm_json_path = os.path.join(llm_out_dir, os.path.basename(file_path) + ".json")

    # 若已生成过，就直接加载
    if os.path.exists(llm_json_path):
        with open(llm_json_path, "r", encoding="utf-8") as f:
            llm_ast = json.load(f)
        print(f"[LLM AST] (cached) => {llm_json_path}")
        return llm_ast

    # 生成
    llm_ast = generate_llm_ast_via_cfg(code)
    with open(llm_json_path, "w", encoding="utf-8") as fout:
        json.dump(llm_ast, fout, indent=4, ensure_ascii=False)
    print(f"[LLM AST] => {llm_json_path}")
    return llm_ast

def process_static_ast(code: str, file_path: str) -> dict:
    """处理静态AST生成和保存"""
    ts_ast = generate_tree_sitter_ast(code)
    ts_out_dir = "static_ast"
    os.makedirs(ts_out_dir, exist_ok=True)
    ts_json_path = os.path.join(ts_out_dir, os.path.basename(file_path) + ".json")
    with open(ts_json_path, "w", encoding="utf-8") as fout:
        json.dump(ts_ast, fout, indent=4, ensure_ascii=False)
    print(f"[TS AST] => {ts_json_path}")
    return ts_ast

def process_single_file(file_path: str):
    """
    1) read code
    2) generate LLM AST (CFG+partial block approach)
    3) generate static AST (tree-sitter)
    4) optionally compare snippet-level label
    5) store JSON
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            code = f.read()
    except Exception as e:
        print(f"[Error] reading {file_path}: {e}")
        return

    print(f"[Processing] {file_path}")

    # 2) LLM AST
    llm_ast = process_llm_ast(code, file_path)

    # 3) tree-sitter AST
    ts_ast = process_static_ast(code, file_path)

    # 4) compare label if needed
    # compare_ast_nodes(llm_ast, ts_ast)

    print(f"[Processing done] {file_path}")

###############################################################################
#                                 main()                                      #
###############################################################################
def main():
    source_dir = "../../dataset/python/"
    if not os.path.isdir(source_dir):
        print(f"[Error] Directory {source_dir} does not exist.")
        return

    # process_single_file(os.path.join(source_dir, "11.py"))
    # 示例：只处理前 10 个 .py 文件
    files = [f for f in os.listdir(source_dir) if f.endswith(".py")]

    with concurrent.futures.ThreadPoolExecutor(max_workers=cpu_count()) as executor:
        futures = []
        pbar = tqdm(total=len(files), desc="处理文件")
        for fname in files:
            full_path = os.path.join(source_dir, fname)
            future = executor.submit(process_single_file, full_path)
            future.add_done_callback(lambda _: pbar.update(1))
            futures.append(future)
        concurrent.futures.wait(futures)
        pbar.close()

if __name__ == "__main__":
    main()


处理文件:   1%|          | 2/210 [00:00<00:17, 12.10it/s]

[Processing] ../../dataset/python/11.py
[Processing] ../../dataset/python/180.py
[Processing] ../../dataset/python/195.py
[Processing] ../../dataset/python/60.py
[LLM AST] (cached) => llm_ast/cfg_block/11.py.json
[Processing] ../../dataset/python/202.py
[Processing] ../../dataset/python/208.py
[TS AST] => static_ast/11.py.json
[Processing done] ../../dataset/python/11.py
[Processing] ../../dataset/python/176.py
[Processing] ../../dataset/python/98.py
[Processing] ../../dataset/python/151.py
[Processing] ../../dataset/python/120.py
[Processing] ../../dataset/python/13.py
[Processing] ../../dataset/python/71.py
[Processing] ../../dataset/python/12.py
[Processing] ../../dataset/python/167.py
[LLM AST] (cached) => llm_ast/cfg_block/167.py.json
[Processing] ../../dataset/python/186.py
[TS AST] => static_ast/167.py.json
[Processing done] ../../dataset/python/167.py
[Processing] ../../dataset/python/14.py
[LLM AST] (cached) => llm_ast/cfg_block/14.py.json
[Processing] ../../dataset/python/62.

处理文件:   2%|▏         | 5/210 [00:00<00:14, 14.29it/s]

[Processing] ../../dataset/python/75.py
[LLM AST] (cached) => llm_ast/cfg_block/9.py.json
[Processing] ../../dataset/python/107.py
[Processing] ../../dataset/python/201.py
[Processing] ../../dataset/python/148.py
[Processing] ../../dataset/python/54.py
[LLM AST] (cached) => llm_ast/cfg_block/75.py.json
[Processing] ../../dataset/python/160.py
[Processing] ../../dataset/python/123.py
[Processing] ../../dataset/python/174.py
[Processing] ../../dataset/python/159.py
[TS AST] => static_ast/9.py.json
[Processing done] ../../dataset/python/9.py
[TS AST] => static_ast/75.py.json
[Processing done] ../../dataset/python/75.py
[LLM AST] (cached) => llm_ast/cfg_block/159.py.json
[Processing] ../../dataset/python/29.py
[Processing] ../../dataset/python/55.py
[Processing] ../../dataset/python/100.py
[Processing] ../../dataset/python/116.py
[LLM AST] (cached) => llm_ast/cfg_block/29.py.json
[TS AST] => static_ast/29.py.json
[Processing done] ../../dataset/python/29.py
[Processing] ../../dataset/pytho

处理文件:   7%|▋         | 14/210 [00:00<00:10, 19.28it/s]

...

[TS AST] => static_ast/32.py.json
[Processing done] ../../dataset/python/32.py
[LLM AST] => llm_ast/cfg_block/186.py.json


处理文件: 100%|█████████▉| 209/210 [28:39<01:28, 88.70s/it]

[TS AST] => static_ast/186.py.json
[Processing done] ../../dataset/python/186.py


处理文件: 100%|██████████| 210/210 [29:39<00:00,  8.47s/it]

[LLM AST] => llm_ast/cfg_block/38.py.json
[TS AST] => static_ast/38.py.json
[Processing done] ../../dataset/python/38.py
